In [26]:
import sympy as sp
import numpy as np

In [49]:
class EKF:
    
    def __init__(self, x0, P0, Q, R, updateX, updateY, JacX, JacY):
        self.X_prev = x0
        self.P = P0
        self.Q = Q
        self.R = R
        self.updateX = updateX
        self.updateY = updateY
        self.JacX = JacX
        self.JacY = JacY
    

In [50]:
q0, q1, q2, q3 = sp.symbols(r"q_{0} q_{1} q_{2} q_{3}")
pitch, roll, yaw = sp.symbols(r" \theta \phi \psi")
p, q, r = sp.symbols("p q r")
dt = sp.symbols(r"\delta_{T}")

Bx, By, Bz = sp.symbols(r"B_{x} B_{y} B_{z}")

X = sp.Matrix([q0, q1, q2, q3])
U = sp.Matrix([p, q, r])

g = sp.Matrix([0, 0, 1])
B = sp.Matrix([Bx, By, Bz])

In [70]:
def skewSym(U):
    p, q, r = U[0], U[1], U[2]
    w = sp.Matrix([[0, -p, -q, -r],
                   [p, 0, r, -q],
                   [q, -r, 0, p],
                   [r, q, -p, 0]])
    return w

def updateX(X, U):
    X_next = 0.5 * skewSym(U) @ X
    X_next = sp.simplify(X_next)
    return X_next
    
def JacX(X, U):
    X_next = X + updateX(X, U) * dt
    jacX = X_next.jacobian(X) 
    jacX = sp.simplify(jacX)
    return jacX

def quatDCM(X):
    q0, q1, q2, q3 = X
    DCM = sp.zeros(3,3)
    DCM[0,0] = q0*q0 + q1*q1 - q2*q2 - q3*q3
    DCM[0,1] = 2 * (q1*q2 + q0*q3)
    DCM[0,2] = 2 * (q1*q3 - q0*q2)
    DCM[1,0] = 2 * (q1*q2 - q0*q3)
    DCM[1,1] = q0*q0 - q1*q1 + q2*q2 - q3*q3
    DCM[1,2] = 2 * (q2*q3 + q0*q1)
    DCM[2,0] = 2 * (q1*q3 + q0*q2)
    DCM[2,1] = 2 * (q2*q3 - q0*q1)
    DCM[2,2] = q0*q0 - q1*q1 - q2*q2 + q3*q3
    return DCM

def updateY(X, U):
    Y = sp.zeros(6,1)
    DCM = quat(X)
    Y[0:3, 0] = DCM @ g
    Y[3:, 0] = DCM @ B
    return Y

def JacY(X, U):
    Y = updateY(X, U)
    jacY = Y.jacobian(X)
    return sp.simplify(jacY)

In [71]:
def Ry(x):
    return sp.Matrix([[ sp.cos(x), 0,  sp.sin(x)],
                      [ 0,         1,  0        ],
                      [-sp.sin(x), 0,  sp.cos(x)]])

def Rx(x):
    return sp.Matrix([[1,          0,         0],
                      [0,  sp.cos(x), -sp.sin(x)],
                      [0,  sp.sin(x),  sp.cos(x)]])

def Rz(x):
    return sp.Matrix([[ sp.cos(x), -sp.sin(x), 0],
                      [ sp.sin(x),  sp.cos(x), 0],
                      [0,       0,            1]])

In [72]:
def quat2euler(q):
    q0, q1, q2, q3 = q
    
    phi = np.arctan2(2 * (q0*q1 + q2*q3), 1 - 2*(q1*q1 + q2*q2))
    theta = np.arcsin(2 * (q0*q2 - q3*q1))
    psi = np.arctan2(2 * (q0*q3 + q1*q2), 1 - 2*(q2*q2 + q3*q3))
    
    return np.array([phi, theta, psi])

def euler2quat(angles):
    roll, pitch, yaw = angles
    
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)
    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)
    
    q = np.zeros(4)
    
    q[0] = cr * cp * cy + sr * sp * sy;
    q[1] = sr * cp * cy - cr * sp * sy;
    q[2] = cr * sp * cy + sr * cp * sy;
    q[3] = cr * cp * sy - sr * sp * cy;
    
    return q

In [87]:
eulDCM = (Rz(yaw) @ Ry(pitch) @ Rx(roll)).transpose()
quatDCM = calcDCM(X)

eulDCM

Matrix([
[                                cos(\psi)*cos(\theta),                                  sin(\psi)*cos(\theta),          -sin(\theta)],
[sin(\phi)*sin(\theta)*cos(\psi) - sin(\psi)*cos(\phi),  sin(\phi)*sin(\psi)*sin(\theta) + cos(\phi)*cos(\psi), sin(\phi)*cos(\theta)],
[sin(\phi)*sin(\psi) + sin(\theta)*cos(\phi)*cos(\psi), -sin(\phi)*cos(\psi) + sin(\psi)*sin(\theta)*cos(\phi), cos(\phi)*cos(\theta)]])

In [84]:
q = np.array([0.5, -0.2, 0.6, 0.1])
q /= np.linalg.norm(q)
eul = quat2euler(q)

print(eulDCM(*eul))
print(quatDCM(q))

[[-0.12121212 -0.21212121 -0.96969697]
 [-0.51515152  0.84848485 -0.12121212]
 [ 0.84848485  0.48484848 -0.21212121]]
[[-0.12121212 -0.21212121 -0.96969697]
 [-0.51515152  0.84848485 -0.12121212]
 [ 0.84848485  0.48484848 -0.21212121]]
